In [1]:
!git clone https://github.com/mindslab-ai/univnet.git
%cd univnet
!pip install -r requirements.txt

Cloning into 'univnet'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 188 (delta 20), reused 17 (delta 17), pack-reused 156 (from 1)
Receiving objects: 100% (188/188), 23.22 MiB | 34.82 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/kaggle/working/univnet
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 40.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 95.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 39.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-

In [2]:
!pip install librosa soundfile numpy

In [3]:
import os
import librosa
import numpy as np
import soundfile as sf

# === 📁 Paths ===
input_dir = "/kaggle/input/directory1"
output_dir = "/kaggle/working/mel_spectrograms"
os.makedirs(output_dir, exist_ok=True)

# === ⚙️ Config ===
target_sr = 22050
n_fft = 1024
hop_length = 256
n_mels = 100

# === 🔁 Process Each Language Folder ===
for language in os.listdir(input_dir):
    lang_in = os.path.join(input_dir, language)
    lang_out = os.path.join(output_dir, language)
    os.makedirs(lang_out, exist_ok=True)

    for file in os.listdir(lang_in):
        if file.endswith(".wav"):
            path = os.path.join(lang_in, file)
            try:
                # === 🎧 Load and Resample ===
                y, sr = sf.read(path)
                if y.ndim > 1:
                    y = y.mean(axis=1)  # Convert stereo to mono

                if sr != target_sr:
                    y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

                # === 🎛️ Compute Linear Mel Spectrogram ===
                mel = librosa.feature.melspectrogram(
                    y=y,
                    sr=target_sr,
                    n_fft=n_fft,
                    hop_length=hop_length,
                    n_mels=n_mels,
                    power=1.0  # Linear magnitude (not dB)
                )

                # === 🔒 Clip to UnivNet Range [-11.5129, 1.2] ===
                mel = np.log(np.clip(mel, a_min=1e-5, a_max=None))  # Natural log
                mel = np.clip(mel, -11.5129, 1.2)

                # === 💾 Save .npy file ===
                save_path = os.path.join(lang_out, file.replace(".wav", ".npy"))
                np.save(save_path, mel)
                print(f"✅ Saved mel: {save_path}")

            except Exception as e:
                print(f"❌ Error processing {file}: {e}")


✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1023.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1102.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1282.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_127.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1024.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1392.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1386.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1072.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1053.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1083.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1357.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1125.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1249.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1280.npy
✅ Saved mel: /kaggle/working/mel_spectrograms/Punjabi/pa_1211.npy
✅ Saved mel

In [4]:
!mkdir -p checkpoints

In [5]:
!cp /kaggle/input/univnet-checkpoint2/pytorch/default/1/univ_c32_0288.pt ./checkpoints/

In [6]:
!mkdir /kaggle/working/output_singfakes2/


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class KernelPredictor(torch.nn.Module):
    ''' Kernel predictor for the location-variable convolutions'''
    def __init__(
            self,
            cond_channels,
            conv_in_channels,
            conv_out_channels,
            conv_layers,
            conv_kernel_size=3,
            kpnet_hidden_channels=64,
            kpnet_conv_size=3,
            kpnet_dropout=0.0,
            kpnet_nonlinear_activation="LeakyReLU",
            kpnet_nonlinear_activation_params={"negative_slope":0.1},
        ):
        '''
        Args:
            cond_channels (int): number of channel for the conditioning sequence,
            conv_in_channels (int): number of channel for the input sequence,
            conv_out_channels (int): number of channel for the output sequence,
            conv_layers (int): number of layers
        '''
        super().__init__()

        self.conv_in_channels = conv_in_channels
        self.conv_out_channels = conv_out_channels
        self.conv_kernel_size = conv_kernel_size
        self.conv_layers = conv_layers

        kpnet_kernel_channels = conv_in_channels * conv_out_channels * conv_kernel_size * conv_layers   # l_w
        kpnet_bias_channels = conv_out_channels * conv_layers                                           # l_b

        self.input_conv = nn.Sequential(
            nn.utils.weight_norm(nn.Conv1d(cond_channels, kpnet_hidden_channels, 5, padding=2, bias=True)),
            getattr(nn, kpnet_nonlinear_activation)(**kpnet_nonlinear_activation_params),
        )

        self.residual_convs = nn.ModuleList()
        padding = (kpnet_conv_size - 1) // 2
        for _ in range(3):
            self.residual_convs.append(
                nn.Sequential(
                    nn.Dropout(kpnet_dropout),
                    nn.utils.weight_norm(nn.Conv1d(kpnet_hidden_channels, kpnet_hidden_channels, kpnet_conv_size, padding=padding, bias=True)),
                    getattr(nn, kpnet_nonlinear_activation)(**kpnet_nonlinear_activation_params),
                    nn.utils.weight_norm(nn.Conv1d(kpnet_hidden_channels, kpnet_hidden_channels, kpnet_conv_size, padding=padding, bias=True)),
                    getattr(nn, kpnet_nonlinear_activation)(**kpnet_nonlinear_activation_params),
                )
            )
        self.kernel_conv = nn.utils.weight_norm(
            nn.Conv1d(kpnet_hidden_channels, kpnet_kernel_channels, kpnet_conv_size, padding=padding, bias=True))
        self.bias_conv = nn.utils.weight_norm(
            nn.Conv1d(kpnet_hidden_channels, kpnet_bias_channels, kpnet_conv_size, padding=padding, bias=True))
        
    def forward(self, c):
        '''
        Args:
            c (Tensor): the conditioning sequence (batch, cond_channels, cond_length)
        '''
        batch, _, cond_length = c.shape
        c = self.input_conv(c)
        for residual_conv in self.residual_convs:
            residual_conv.to(c.device)
            c = c + residual_conv(c)
        k = self.kernel_conv(c)
        b = self.bias_conv(c)
        kernels = k.contiguous().view(
            batch,
            self.conv_layers,
            self.conv_in_channels,
            self.conv_out_channels,
            self.conv_kernel_size,
            cond_length,
        )
        bias = b.contiguous().view(
            batch,
            self.conv_layers,
            self.conv_out_channels,
            cond_length,
        )

        return kernels, bias

    def remove_weight_norm(self):
        nn.utils.remove_weight_norm(self.input_conv[0])
        nn.utils.remove_weight_norm(self.kernel_conv)
        nn.utils.remove_weight_norm(self.bias_conv)
        for block in self.residual_convs:
            nn.utils.remove_weight_norm(block[1])
            nn.utils.remove_weight_norm(block[3])

class LVCBlock(torch.nn.Module):
    '''the location-variable convolutions'''
    def __init__(
            self,
            in_channels,
            cond_channels,
            stride,
            dilations=[1, 3, 9, 27],
            lReLU_slope=0.2,
            conv_kernel_size=3,
            cond_hop_length=256,
            kpnet_hidden_channels=64,
            kpnet_conv_size=3,
            kpnet_dropout=0.0,
        ):
        super().__init__()

        self.cond_hop_length = cond_hop_length
        self.conv_layers = len(dilations)
        self.conv_kernel_size = conv_kernel_size

        self.kernel_predictor = KernelPredictor(
            cond_channels=cond_channels,
            conv_in_channels=in_channels,
            conv_out_channels=2 * in_channels, 
            conv_layers=len(dilations),
            conv_kernel_size=conv_kernel_size,
            kpnet_hidden_channels=kpnet_hidden_channels,
            kpnet_conv_size=kpnet_conv_size,
            kpnet_dropout=kpnet_dropout,
            kpnet_nonlinear_activation_params={"negative_slope":lReLU_slope}
        )
        
        self.convt_pre = nn.Sequential(
            nn.LeakyReLU(lReLU_slope),
            nn.utils.weight_norm(nn.ConvTranspose1d(in_channels, in_channels, 2 * stride, stride=stride, padding=stride // 2 + stride % 2, output_padding=stride % 2)),
        )
        
        self.conv_blocks = nn.ModuleList()
        for dilation in dilations:
            self.conv_blocks.append(
                nn.Sequential(
                    nn.LeakyReLU(lReLU_slope),
                    nn.utils.weight_norm(nn.Conv1d(in_channels, in_channels, conv_kernel_size, padding=dilation * (conv_kernel_size - 1) // 2, dilation=dilation)),
                    nn.LeakyReLU(lReLU_slope),
                )
            )

    def forward(self, x, c):
        ''' forward propagation of the location-variable convolutions.
        Args: 
            x (Tensor): the input sequence (batch, in_channels, in_length)
            c (Tensor): the conditioning sequence (batch, cond_channels, cond_length)
        
        Returns:
            Tensor: the output sequence (batch, in_channels, in_length)
        ''' 
        _, in_channels, _ = x.shape         # (B, c_g, L')
        
        x = self.convt_pre(x)               # (B, c_g, stride * L')
        kernels, bias = self.kernel_predictor(c)

        for i, conv in enumerate(self.conv_blocks):
            output = conv(x)                # (B, c_g, stride * L')
            
            k = kernels[:, i, :, :, :, :]   # (B, 2 * c_g, c_g, kernel_size, cond_length)
            b = bias[:, i, :, :]            # (B, 2 * c_g, cond_length)

            output = self.location_variable_convolution(output, k, b, hop_size=self.cond_hop_length)    # (B, 2 * c_g, stride * L'): LVC
            x = x + torch.sigmoid(output[ :, :in_channels, :]) * torch.tanh(output[:, in_channels:, :]) # (B, c_g, stride * L'): GAU
        
        return x
    
    def location_variable_convolution(self, x, kernel, bias, dilation=1, hop_size=256):
        ''' perform location-variable convolution operation on the input sequence (x) using the local convolution kernl. 
        Time: 414 μs ± 309 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each), test on NVIDIA V100. 
        Args:
            x (Tensor): the input sequence (batch, in_channels, in_length). 
            kernel (Tensor): the local convolution kernel (batch, in_channel, out_channels, kernel_size, kernel_length) 
            bias (Tensor): the bias for the local convolution (batch, out_channels, kernel_length) 
            dilation (int): the dilation of convolution. 
            hop_size (int): the hop_size of the conditioning sequence. 
        Returns:
            (Tensor): the output sequence after performing local convolution. (batch, out_channels, in_length).
        '''
        batch, _, in_length = x.shape
        batch, _, out_channels, kernel_size, kernel_length = kernel.shape
        assert in_length == (kernel_length * hop_size), "length of (x, kernel) is not matched"

        padding = dilation * int((kernel_size - 1) / 2)
        x = F.pad(x, (padding, padding), 'constant', 0)     # (batch, in_channels, in_length + 2*padding)
        x = x.unfold(2, hop_size + 2 * padding, hop_size)   # (batch, in_channels, kernel_length, hop_size + 2*padding)

        if hop_size < dilation:
            x = F.pad(x, (0, dilation), 'constant', 0)
        x = x.unfold(3, dilation, dilation)     # (batch, in_channels, kernel_length, (hop_size + 2*padding)/dilation, dilation)
        x = x[:, :, :, :, :hop_size]          
        x = x.transpose(3, 4)                   # (batch, in_channels, kernel_length, dilation, (hop_size + 2*padding)/dilation)  
        x = x.unfold(4, kernel_size, 1)         # (batch, in_channels, kernel_length, dilation, _, kernel_size)

        o = torch.einsum('bildsk,biokl->bolsd', x, kernel)
        o = o.to(memory_format=torch.channels_last_3d)
        bias = bias.unsqueeze(-1).unsqueeze(-1).to(memory_format=torch.channels_last_3d)
        o = o + bias
        o = o.contiguous().view(batch, out_channels, -1)

        return o

    def remove_weight_norm(self):
        self.kernel_predictor.remove_weight_norm()
        nn.utils.remove_weight_norm(self.convt_pre[1])
        for block in self.conv_blocks:
            nn.utils.remove_weight_norm(block[1])

In [8]:
import torch
import torch.nn as nn
from omegaconf import OmegaConf

# from .lvcnet import LVCBlock

MAX_WAV_VALUE = 32768.0

class Generator(nn.Module):
    """UnivNet Generator"""
    def __init__(self, hp):
        super(Generator, self).__init__()
        # Use the mel_channels from config
        self.mel_channel = hp.audio.n_mel_channels
        print(f"✅ mel_channel set to: {self.mel_channel}")
        
        # Rest of the initialization remains the same...
        self.noise_dim = hp.gen.noise_dim
        self.hop_length = hp.audio.hop_length
        channel_size = hp.gen.channel_size
        kpnet_conv_size = hp.gen.kpnet_conv_size

        self.res_stack = nn.ModuleList()
        hop_length = 1
        for stride in hp.gen.strides:
            hop_length = stride * hop_length
            self.res_stack.append(
                LVCBlock(
                    channel_size,
                    hp.audio.n_mel_channels,
                    stride=stride,
                    dilations=hp.gen.dilations,
                    lReLU_slope=hp.gen.lReLU_slope,
                    cond_hop_length=hop_length,
                    kpnet_conv_size=kpnet_conv_size
                )
            )
        
        self.conv_pre = \
            nn.utils.weight_norm(nn.Conv1d(hp.gen.noise_dim, channel_size, 7, padding=3, padding_mode='reflect'))

        self.conv_post = nn.Sequential(
            nn.LeakyReLU(hp.gen.lReLU_slope),
            nn.utils.weight_norm(nn.Conv1d(channel_size, 1, 7, padding=3, padding_mode='reflect')),
            nn.Tanh(),
        )

    def forward(self, c, z):
        '''
        Args: 
            c (Tensor): the conditioning sequence of mel-spectrogram (batch, mel_channels, in_length) 
            z (Tensor): the noise sequence (batch, noise_dim, in_length)
        
        '''
        z = self.conv_pre(z)                # (B, c_g, L)

        for res_block in self.res_stack:
            res_block.to(z.device)
            z = res_block(z, c)             # (B, c_g, L * s_0 * ... * s_i)

        z = self.conv_post(z)               # (B, 1, L * 256)

        return z

    def eval(self, inference=False):
        super(Generator, self).eval()
        # don't remove weight norm while validation in training loop
        if inference:
            self.remove_weight_norm()

    def remove_weight_norm(self):
        print('Removing weight norm...')

        nn.utils.remove_weight_norm(self.conv_pre)

        for layer in self.conv_post:
            if len(layer.state_dict()) != 0:
                nn.utils.remove_weight_norm(layer)

        for res_block in self.res_stack:
            res_block.remove_weight_norm()

    def inference(self, c, z=None):
        # pad input mel with zeros to cut artifact
        # see https://github.com/seungwonpark/melgan/issues/8
        zero = torch.full((1, self.mel_channel, 10), -11.5129).to(c.device)
        mel = torch.cat((c, zero), dim=2)
        
        if z is None:
            z = torch.randn(1, self.noise_dim, mel.size(2)).to(mel.device)

        audio = self.forward(mel, z)
        audio = audio.squeeze() # collapse all dimension except time axis
        audio = audio[:-(self.hop_length*10)]
        audio = MAX_WAV_VALUE * audio
        audio = audio.clamp(min=-MAX_WAV_VALUE, max=MAX_WAV_VALUE-1)
        audio = audio.short()

        return audio

if __name__ == '__main__':
    hp = OmegaConf.load('/kaggle/working/univnet/config/default_c32.yaml')
    model = Generator(hp)

    c = torch.randn(3, 100, 10)
    z = torch.randn(3, 64, 10)
    print(c.shape)

    y = model(c, z)
    print(y.shape)
    assert y.shape == torch.Size([3, 1, 2560])

    pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(pytorch_total_params)

✅ mel_channel set to: 100


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


torch.Size([3, 100, 10])
torch.Size([3, 1, 2560])
14865506


In [9]:
import os
import glob
import torch
import numpy as np
from tqdm import tqdm
from scipy.io.wavfile import write
from omegaconf import OmegaConf

# === 🔧 Set your paths here ===
checkpoint_path = "./checkpoints/univ_c32_0288.pt"
input_folder = "/kaggle/working/mel_spectrograms"
output_folder = "/kaggle/working/output_singfakes2"
os.makedirs(output_folder, exist_ok=True)

# === 📦 Load checkpoint and config ===
checkpoint = torch.load(checkpoint_path, map_location='cuda' if torch.cuda.is_available() else 'cpu')
hp = OmegaConf.create(checkpoint['hp_str'])

# === 🧠 Update mel_channels to match checkpoint (100) ===
# The checkpoint was trained with 100 mel channels, so we need to use that
hp.mel_channels = 100  # Changed from 80 to match checkpoint
hp.audio.n_mel_channels = 100  # Changed from 80 to match checkpoint

# === 🧠 Load UnivNet Generator with correct mel_channels ===
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Generator(hp).to(device)

# === 🔁 Load state dict (handle 'module.' prefix) ===
new_state_dict = {}
for k, v in checkpoint['model_g'].items():
    if k.startswith('module.'):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v
        
# Load the state dict strictly to catch any other mismatches
model.load_state_dict(new_state_dict, strict=True)
model.eval(inference=True)

# === 🚀 Run inference for all language folders ===
for language in os.listdir(input_folder):
    lang_input_path = os.path.join(input_folder, language)
    lang_output_path = os.path.join(output_folder, language)
    os.makedirs(lang_output_path, exist_ok=True)

    mel_files = glob.glob(os.path.join(lang_input_path, "*.npy"))

    with torch.no_grad():
        for melpath in tqdm(mel_files, desc=f"🎵 Processing {language}"):
            mel = np.load(melpath)
            if mel.ndim == 2:
                mel = mel[np.newaxis, :, :]  # (1, mel_bins, frames)
                
            # Pad or interpolate mel to 100 channels if needed
            if mel.shape[1] != 100:
                if mel.shape[1] < 100:
                    # Pad with zeros if input has fewer channels
                    pad = np.zeros((mel.shape[0], 100 - mel.shape[1], mel.shape[2]))
                    mel = np.concatenate([mel, pad], axis=1)
                else:
                    # Truncate if input has more channels
                    mel = mel[:, :100, :]

            mel_tensor = torch.tensor(mel).float().to(device)

            audio = model.inference(mel_tensor)
            audio = audio.squeeze().cpu().numpy()

            basename = os.path.basename(melpath).replace(".npy", f"_ep{checkpoint['epoch']:04d}.wav")
            out_path = os.path.join(lang_output_path, basename)
            write(out_path, hp.audio.sampling_rate, audio)
            print(f"✅ Saved: {out_path}")

✅ mel_channel set to: 100
Removing weight norm...


🎵 Processing Italian:   2%|▏         | 8/500 [00:00<00:32, 14.98it/s]

✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1398_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1081_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1197_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1245_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1335_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_113_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1205_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1200_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1293_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1026_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1439_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1124_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1350_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1246_ep0288.wav


🎵 Processing Italian:   4%|▍         | 22/500 [00:00<00:12, 37.06it/s]

✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1371_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_104_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1096_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_125_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_111_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1206_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_129_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1038_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_118_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1409_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1184_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1024_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_1183_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/Italian/it_144_ep0288.wav


...

✅ Saved: /kaggle/working/output_singfakes2/German/de_1347_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1234_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1191_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1343_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1095_ep0288.wav


🎵 Processing German:  99%|█████████▉| 494/500 [00:07<00:00, 70.06it/s]

✅ Saved: /kaggle/working/output_singfakes2/German/de_1187_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1364_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1380_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1433_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_134_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1379_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1337_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1391_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1218_ep0288.wav
✅ Saved: /kaggle/working/output_singfakes2/German/de_1040_ep0288.wav


🎵 Processing German: 100%|██████████| 500/500 [00:07<00:00, 68.88it/s]

✅ Saved: /kaggle/working/output_singfakes2/German/de_1445_ep0288.wav


In [10]:
import shutil

# Folder to zip
source_folder = "/kaggle/working/output_singfakes2"
zip_output_path = "/kaggle/working/output_singfakes.zip"

# Create zip
shutil.make_archive(zip_output_path.replace(".zip", ""), 'zip', source_folder)

print(f"✅ Zipped successfully: {zip_output_path}")


✅ Zipped successfully: /kaggle/working/output_singfakes.zip
